In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import os, sys
import re, string
from collections import Counter
import csv

In [3]:
!pip install verstack

  Created wheel for verstack: filename=verstack-0.3.1-cp36-none-any.whl size=14343 sha256=621fbdd6aa3f0d9570e9afaff25617950bdef180ee8f0eac7c6ba98942a31ece
  Stored in directory: /root/.cache/pip/wheels/15/1b/58/10e59516150cea4d9b1dbacceb3bffcc0cfd2d166efabec2f6
Successfully built verstack


In [4]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit 

In [5]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 20.0MB/s 
     |████████████████████████████████| 317kB 24.1MB/s 
     |████████████████████████████████| 7.4MB 45.5MB/s 
     |████████████████████████████████| 1.8MB 45.9MB/s 
     |████████████████████████████████| 2.9MB 56.9MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 1.3MB 47.8MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 112kB 54.7MB/s 
     |████████████████████████████████| 133kB 45.3MB/s 
     |████████████████████████████████| 4.5MB 43.0MB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
     |████████████████████████████████| 163kB 51.8MB/s 
     |████████████████████████████████| 6.9MB 44.4MB/s 
     |████████████████████████████████| 102kB 16.0MB/s 
     |████████████████████████████████| 133kB 54.9MB/s 
     |████████████████████████████████| 890kB 44.8MB/s 
     |████████████████████████████████| 1.1MB 44.2M

In [6]:
# Importing the libraries needed
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [7]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

In [8]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [108]:
path = "/content/drive/My Drive/"
path_data_train = path + "/covidChallenge/data/train.csv"

In [109]:
train = pd.read_csv(path_data_train)

In [110]:
train.columns

Index(['id', 'timestamp', 'retweet_count', 'user_verified',
       'user_statuses_count', 'user_followers_count', 'user_friends_count',
       'user_mentions', 'urls', 'hashtags', 'text'],
      dtype='object')

In [111]:
counter_user_mentions = train['user_mentions'].isnull().sum()
counter_urls = train['urls'].isnull().sum()
counter_hashtags = train['hashtags'].isnull().sum()    

In [112]:
print("\t User_mentions column NULL entries: ", counter_user_mentions)
print("\t Urls column column NULL entries: ", counter_urls)
print("\t Hashtags column NULL entries: ", counter_hashtags)
print("\t Number of row: ", len(train.index))

	 User_mentions column NULL entries:  611486
	 Urls column column NULL entries:  451697
	 Hashtags column NULL entries:  593326
	 Number of row:  665777


**Comment:**

- > As noted in the previous cell, we can drop the columns since most of the entries does not provide much information

In [113]:
train.drop('user_mentions', axis=1, inplace=True)
train.drop('urls', axis=1, inplace=True)
train.drop('hashtags', axis=1, inplace=True)

In [114]:
train[:10]

,id,timestamp,retweet_count,user_verified,user_statuses_count,user_followers_count,user_friends_count,text
0,0,1588696955143,0,False,68460,1101,1226,Smh I give up
1,1,1588464948124,0,False,309,51,202,"Most of us are Human Beings, but I think you m..."
2,2,1588634673360,0,False,3241,1675,2325,"Old dirty tricks Trump, at it again...like we ..."
3,3,1588433158672,0,False,32327,667,304,Seriously..... I worked 86 hours my last check...
4,4,1588582751599,0,False,581,42,127,May ALMIGHTY ALLAH have mercy on us all. Only ...
5,5,1588434563287,0,False,7214,503,1126,They couldn’t care less.
6,6,1588692966869,2,False,372,738,472,Extremely valid points being made here 👇🏾 #Eth...
7,7,1588316892450,1,False,2085,3808,153,COVID-19 dominated the discussion Tuesday at a...
8,8,1588625905286,0,False,17765,11666,40,BC now has 112 patients on ventilators. 17 of...
9,9,1588604315931,0,False,3086,66,241,a COVID-19 vaccine would be pretty lit ngl


In [115]:
!pip install emoji

In [116]:
import emoji
# Some basic helper functions to clean text by removing urls, emojis, html tags and punctuations.

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_emoji(text):
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
    """
    #return re.sub('[^A-Za-z0-9]+', ' ', text)
    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)
    return new_text


def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

In [117]:
# Applying helper functions
train_clean = train.copy()
train_clean['text'] = train['text'].apply(lambda x: remove_URL(x))
train_clean['text'] = train['text'].apply(lambda x: remove_emoji(x))
train_clean['text'] = train['text'].apply(lambda x: remove_html(x))
train_clean['text'] = train['text'].apply(lambda x: remove_punct(x))

In [118]:
train_clean.head()

,id,timestamp,retweet_count,user_verified,user_statuses_count,user_followers_count,user_friends_count,text
0,0,1588696955143,0,False,68460,1101,1226,Smh I give up
1,1,1588464948124,0,False,309,51,202,Most of us are Human Beings but I think you mi...
2,2,1588634673360,0,False,3241,1675,2325,Old dirty tricks Trump at it againlike we dont...
3,3,1588433158672,0,False,32327,667,304,Seriously I worked 86 hours my last check and ...
4,4,1588582751599,0,False,581,42,127,May ALMIGHTY ALLAH have mercy on us all Only l...


In [119]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [120]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.text[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        #print(ids)
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.retweet_count[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [121]:
train_clean = train_clean[['text','retweet_count']]

In [122]:
train_clean[:15]

,text,retweet_count
0,Smh I give up,0
1,Most of us are Human Beings but I think you mi...,0
2,Old dirty tricks Trump at it againlike we dont...,0
3,Seriously I worked 86 hours my last check and ...,0
4,May ALMIGHTY ALLAH have mercy on us all Only l...,0
5,They couldn’t care less,0
6,Extremely valid points being made here 👇🏾 Ethi...,2
7,COVID19 dominated the discussion Tuesday at a ...,1
8,BC now has 112 patients on ventilators 17 of ...,0
9,a COVID19 vaccine would be pretty lit ngl,0


In [123]:
train_clean.retweet_count[638928]

2

In [124]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset= train_clean.sample(frac=train_size,random_state=200)
test_dataset= train_clean.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(train_clean.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (665777, 2)
TRAIN Dataset: (532622, 2)
TEST Dataset: (133155, 2)


In [125]:
train_dataset.retweet_count

0          0
1          0
2          0
3          0
4          0
          ..
532617    89
532618     0
532619     0
532620     0
532621     0
Name: retweet_count, Length: 532622, dtype: int64

In [126]:
training_set.__getitem__(0)

{'ids': tensor([  101,  2543,  4689, 26311,  1111, 13937, 12473,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [127]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [128]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.regressor = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.regressor(pooler)
        return output

In [129]:
model = DistillBERTClass()
model.to(device)

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_feat

In [130]:
path_data_eval = path + "/covidChallenge/data/evaluation.csv"
evaluation = pd.read_csv(path_data_eval)

In [131]:
evaluation.shape

(285334, 10)

In [132]:
# Saving the files for re-use

output_model_file = path + 'covidChallenge/pytorch_distilbert_news.bin'
output_vocab_file = path + 'covidChallenge/vocab_distilbert_news.bin'

In [133]:
train = True
if not train:
  #model.load_state_dict(torch.load(output_model_file))
  model = torch.load(output_model_file)
  #model_vocab = torch.load(output_vocab_file)
  model.eval()

In [134]:
# Creating the loss function and optimizer
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [135]:
# Function to calcuate the accuracy of the model

def calcuate_accu(predicted, targets):
    n_correct = (predicted==targets).sum().item()
    return n_correct

In [136]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _, data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        #outputs = torch.tensor(outputs, dtype=torch.cuda.long)
        #print("Outputs", outputs.shape, ", Type:", type(outputs))
        #print("targets", targets.shape, ", Type:", type(targets.T))
        loss = loss_function(outputs[:,0].float(), targets.float())
        tr_loss += loss.item()
        #big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(outputs, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")
            print(f"Loss: ", {loss})
            targets_cpu, outputs_cpu = targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()
            print("MAE Prediction error: ", mean_absolute_error(y_true=targets_cpu, y_pred=outputs_cpu))

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [137]:
for epoch in range(EPOCHS):
  train(epoch)

Training Loss per 5000 steps: 4.454490661621094
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(4.4545, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  1.1982222944498062
Training Loss per 5000 steps: 11316785.849936096
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(330.5974, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  18.10863733291626
Training Loss per 5000 steps: 8322352.053500893
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(1207.1611, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  34.73920822143555
Training Loss per 5000 steps: 9101063.322980314
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(3047.2744, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  47.98824916779995
Training Loss per 5000 steps: 8313364.343863686
Training Accuracy per 5000 steps: 0.0
Loss:  {tensor(6083.8208, device='cuda:0', grad_fn=<MseLossBackward>)}
MAE Prediction error:  76.57675170898438
Training Los

In [138]:
def valid(model, testing_loader):
    model.eval()
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    n_correct = 0; n_wrong = 0; total = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs.float(), targets.float())
            
            tr_loss += loss.item()
            #big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(outputs, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
                print("Loss: ", loss)
                targets_cpu, outputs_cpu = targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()
                print("MAE Prediction error: ", mean_absolute_error(y_true=targets_cpu, y_pred=outputs_cpu))
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    #return epoch_accu
    return epoch_accu

In [139]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

Validation Loss per 100 steps: 1427.5477294921875
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(1427.5477, device='cuda:0')
MAE Prediction error:  37.78282165527344
Validation Loss per 100 steps: 6060764.270711327
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(1477.6577, device='cuda:0')
MAE Prediction error:  38.43378829956055
Validation Loss per 100 steps: 4919099.802426118
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(1495.7825, device='cuda:0')
MAE Prediction error:  38.666568756103516
Validation Loss per 100 steps: 4320901.170377078
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(1477.6729, device='cuda:0')
MAE Prediction error:  38.43575859069824
Validation Loss per 100 steps: 3748770.9179120115
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(8340.2822, device='cuda:0')
MAE Prediction error:  81.40761375427246
Validation Loss per 100 steps: 3868862.2235762305
Validation Accuracy per 100 steps: 0.0
Loss:  tensor(12906.9697, device='cuda:0')
MAE Predic

In [146]:
save = True
if save == True:
  model_to_save = model
  torch.save(model_to_save, output_model_file)
  tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed


In [141]:
EVAL_BATCH_SIZE = 1
eval_params = {'batch_size': EVAL_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

In [142]:
class TriageEval(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        title = str(self.data.text[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        #print(ids)
        mask = inputs['attention_mask']
 
        return {
            'id': torch.tensor(self.data.id[index], dtype=torch.long),
            'txt' : self.data.text[index],
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            #'targets': torch.tensor(self.data.retweet_count[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [143]:
evaluation_set = TriageEval(evaluation, tokenizer, MAX_LEN)
evaluationloader = DataLoader(evaluation_set, **eval_params)

In [144]:
evaluation_set.__getitem__(16)

{'id': tensor(665792),
 'ids': tensor([  101,   137,  5331,  8223, 18266,  2107, 24506,  1348, 17261,  1106,
          1129,  1226,  1104,  1216,   170, 14820,  1661,  1104, 24146,   119,
         26010,  6099,  2993,  1359, 11279,  1110,  1184,  2228,  1366,   189,
          5345,  1107,   108,  2508,  1181,  1942, 11252,  1105,  1122,  1110,
          1189,  1155,  1103,  1167,  3110,  1165, 10715,  1929,  1250,  1487,
           119,   108, 11279,   108, 18732, 23314,  2137, 16382, 18630,   131,
           120,   120,   189,   119,  1884,   120,   181,  1665,  4538, 12426,
          1475,  2101,  1181,  1568,  1580,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,

In [145]:
# And then we predict the values for our testing set
with torch.no_grad():
  path_to_save = path + "/covidChallenge/data/predictions.txt"
  f = open(path_to_save,"w+")
  with open(path_to_save, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "NoRetweets"])

    for _, data in enumerate(evaluationloader, 0):
        id = int(data['id'])
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        #targets = data['targets'].to(device, dtype = torch.long)
        #print("targets: ", targets)
        output = model(ids, mask).squeeze()
        
        print("Id: {}, prediction: {}".format(id, int(output)))
        writer.writerow([str(id) , str(int(output))])


#y_pred = model(eval)
#print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=y_pred))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Id: 946110, prediction: 158
Id: 946111, prediction: 144
Id: 946112, prediction: 37
Id: 946113, prediction: 158
Id: 946114, prediction: 38
Id: 946115, prediction: 158
Id: 946116, prediction: 38
Id: 946117, prediction: 38
Id: 946118, prediction: 158
Id: 946119, prediction: 37
Id: 946120, prediction: 158
Id: 946121, prediction: 42
Id: 946122, prediction: 158
Id: 946123, prediction: 37
Id: 946124, prediction: 37
Id: 946125, prediction: 38
Id: 946126, prediction: 37
Id: 946127, prediction: 38
Id: 946128, prediction: 158
Id: 946129, prediction: 158
Id: 946130, prediction: 37
Id: 946131, prediction: 145
Id: 946132, prediction: 42
Id: 946133, prediction: 158
Id: 946134, prediction: 119
Id: 946135, prediction: 158
Id: 946136, prediction: 38
Id: 946137, prediction: 158
Id: 946138, prediction: 37
Id: 946139, prediction: 40
Id: 946140, prediction: 38
Id: 946141, prediction: 37
Id: 946142, prediction: 158
Id: 946143, pred